# Initial exploration of EV charger map resources

In [1]:
import requests
import pandas as pd
import json
from os import listdir


We first take at look at the open source 'openchargemap' 

In [28]:
endpoint =  'https://api.openchargemap.io/v3/poi/?output=json&countrycode=DK&maxresults=1000&key=fe1d8895-1ef6-4b4f-beb5-d95e7253532e'
req = requests.get(endpoint)
resp = json.loads(req.content)
resp[0]

{'DataProvider': {'WebsiteURL': 'http://openchargemap.org',
  'Comments': None,
  'DataProviderStatusType': {'IsProviderEnabled': True,
   'ID': 1,
   'Title': 'Manual Data Entry'},
  'IsRestrictedEdit': False,
  'IsOpenDataLicensed': True,
  'IsApprovedImport': True,
  'License': 'Licensed under Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)',
  'DateLastImported': None,
  'ID': 1,
  'Title': 'Open Charge Map Contributors'},
 'OperatorInfo': {'WebsiteURL': None,
  'Comments': None,
  'PhonePrimaryContact': None,
  'PhoneSecondaryContact': None,
  'IsPrivateIndividual': None,
  'AddressInfo': None,
  'BookingURL': None,
  'ContactEmail': None,
  'FaultReportEmail': None,
  'IsRestrictedEdit': None,
  'ID': 1,
  'Title': '(Unknown Operator)'},
 'UsageType': {'IsPayAtLocation': None,
  'IsMembershipRequired': None,
  'IsAccessKeyRequired': None,
  'ID': 1,
  'Title': 'Public'},
 'StatusType': {'IsOperational': True,
  'IsUserSelectable': True,
  'ID': 50,
  'Tit

In [27]:
values = dict()
values['latitude'] = []
values['longitude'] = []
values['address'] = []
values['plugs'] = []
values['city'] = []
values['post_code'] = []
values['country_code'] = []

for r in resp:
    values['latitude'].append(r['AddressInfo']['Latitude'])
    values['longitude'].append(r['AddressInfo']['Longitude'])
    values['address'].append(r['AddressInfo']['AddressLine1'])
    values['plugs'].append(r['NumberOfPoints'])
    values['city'].append(r['AddressInfo']['Town'])
    values['post_code'].append(r['AddressInfo']['Postcode'])
    values['country_code'].append(r['AddressInfo']['Country']['ISOCode'])

api_chargers = pd.DataFrame(values)
api_chargers




,latitude,longitude,address,plugs,city,post_code,country_code
0,55.366066,10.353847,148 Faaborgvej,2,Odense,5250,DK
1,55.413711,10.341636,2A Hans Egedes Vej,4,Odense,5210,DK
2,55.427823,10.405292,120 Havnegade,2,Odense,5000,DK
3,55.392431,10.435517,2A Cikorievej,2,Odense,5220,DK
4,55.380790,10.428737,Stærmosegårdsvej,2,Killerup,5220,DK
...,...,...,...,...,...,...,...
224,55.709031,12.485076,Astersvej 11,1,Brønshøj,2700,DK
225,55.668536,12.573710,Kalvebod Brygge 5,1,Copenhagen,1560,DK
226,55.682645,12.588667,Bredgade 35,1,København,1260,DK
227,55.381406,10.394466,Hunderupgade 2,1,Odense,5230,DK


We only receive 229 observations which is not sufficient. The following danish ressource states that we have approx 1800 charging stations https://fm.dk/media/18388/delrapport-2_veje-til-en-veludbygget-ladeinfrastruktur_web.pdf.

We will therefore scrape the site 'https://www.plugshare.com/'. They are currently not providing API access for students. Since the content on plugshare is dynamically loaded using their own api we cannot scrape it using simple calls with the requests module. All queries made that way return errors. However, we can use the Chrome Devtools and navigate to the Network tab to see which underlying queries are being made and what result they return. 

Using this approach we make a 'rough sweep' across Denmark and save the query results as txt files in the folder 'charging_stations' which among other things contain the url of each charging station. These we use to initialize a Selenium scraper of the entire Plughshare map of charging stations in Denmark. This scraper is implemented in 'charger_scraper.py

In [ ]:
files = [f for f in listdir('charging_stations/')]
print(files)
obj = []
for fil in files:
    with open('charging_stations/' + fil) as f:
        lines = f.readlines()[0]
        txt = ""
        obj.extend(json.loads(lines))


In [30]:
urls = {'link':[]}
for i in obj:
    urls['link'].append(i['url'])
urls = pd.DataFrame(urls)
urls.to_csv('charger_links.csv')